In [1]:
using DIVAnd
using PyPlot
using NCDatasets
using Missings
using Interpolations

if VERSION >= v"0.7"
    using Random
    using DelimitedFiles
    using Statistics
    using Printf
    using FileIO
else
    using Compat: @info, @warn, range, cat
end

In [2]:
# Module where the output grid and the modules are defined
include("../src/emodnet_bio_grid.jl");
include("../src/make_traits_products.jl".jl");

LoadError: syntax: cannot juxtapose string literal

In [3]:
datafile = "/home/ctroupin/tmp/Emodnet-Bio/Traits/Benthos_Traits_ab.csv"
figdir = "../figures/traits/"
outputdir = "../output/"
outputfile = joinpath(outputdir, replace(basename(datafile), ".csv" => ".nc"))

"../output/Benthos_Traits_ab.nc"

## Read data

In [4]:
if isfile(datafile)
    data,header = readdlm(datafile,',',header = true);
else
    @warn "Data file doesn't exist"
end
varlist = header[5:end];
ntraits = length(varlist);
@info "Working on $(ntraits) traits"

┌ Info: Working on 62 traits
└ @ Main In[4]:8


### Get coordinates

In [5]:
indexlon = findfirst(header .== "x").I[2];
indexlat = findfirst(header .== "y").I[2];
obslon = Vector{Float64}(data[:,indexlon]);
obslat = Vector{Float64}(data[:,indexlat]);

### Mask and bathymetry

In [6]:
# Grid stored in emodnet_bio_grid.jl
xi,yi = DIVAnd.ndgrid(gridlonFish, gridlatFish);

# Mask
topodir = "../data/"
topofile = joinpath(topodir, "gebco_30sec_8.nc");

if isfile(topofile)
    bx, by, b = DIVAnd.load_bath(topofile,true,gridlonBenthos, gridlatBenthos);
    xmask, ymask, mmask = DIVAnd.load_mask(topofile,true,gridlonBenthos, gridlatBenthos,[0]);
else
    @error "Bathymetry file doesn't exist"
end
mmask = mmask[:,:,1]
@info size(mmask)

# Metrics
pm = ones(size(xi)) / (xi[2,1]-xi[1,1]);
pn = ones(size(xi)) / (yi[1,2]-yi[1,1]);

┌ Info: (451, 371)
└ @ Main In[6]:15


## Parameter choice 

In [7]:
# Correlation length
len = 2.;
# Observational error variance normalized by the background error variance
epsilon2 = 5.;

The weight is decreased for very close points.

In [8]:
rdiag=1.0./DIVAnd.weight_RtimesOne((obslon, obslat),(0.03,0.03))
@show maximum(rdiag),mean(rdiag);
epsilon2 = epsilon2*rdiag;

(maximum(rdiag), mean(rdiag)) = (439.41430029504994, 33.49833660679846)


## Analysis
All the fields are stored in a 3D array, to be later written to a netCDF file.

In [9]:
# Allocate matrix with all the gridded fields
trait_all = Array{Float64}(undef, ntraits, length(gridlonFish), length(gridlatFish));
trait_err = Array{Float64}(undef, length(gridlonFish), length(gridlatFish));
for i in 1:ntraits
    @info "Working on $(varlist[i])" 
    indextrait = findfirst(header .== varlist[i]).I[2];
    trait = Vector{Float64}(data[:,indextrait]) 
    traits_interp = traits_analysis(obslon, obslat, trait);
    if i == 1
        @info "Computing error field"
        trais_err = traits_error(obslon, obslat, trait);
    end
    trait_all[i,:,:] = traits_interp;
end

┌ Info: Working on Body.length..1cm
└ @ Main In[9]:5


UndefVarError: UndefVarError: traits_analysis not defined

### Write netCDF

In [10]:
write_traits_nc(outputfile, gridlonFish, gridlatFish, 
                trait_all, traits_err, varlist);

UndefVarError: UndefVarError: write_traits_nc not defined

## Plots

In [11]:
for ii = 1:ntraits
    make_plot_grid(gridlonFish, gridlatFish, trait_all[ii,:,:], varlist[ii]; vmin=0)
    figname = joinpath(figdir, replace(varlist[ii], "." => "_"))
    PyPlot.savefig(figname, dpi=300, bbox_inches="tight")
    PyPlot.close_figs()
end

MethodError: MethodError: no method matching genperm(::Tuple{Base.OneTo{Int64}}, ::Tuple{Int64,Int64})
Closest candidates are:
  genperm(::Tuple{Vararg{Any,N}}, !Matched::Tuple{Vararg{Int64,N}}) where N at permuteddimsarray.jl:77
  genperm(::Any, !Matched::AbstractArray{Int64,1}) at permuteddimsarray.jl:78

In [12]:
figure()
ax1 = subplot(1,1,1)
pcolormesh(gridlonBenthos, gridlatBenthos, permutedims(trait_all[1,:,:], [2,1]), vmin=0, vmax=1.)
PyPlot.plot(obslon, obslat, "ro", markersize=0.1)
PyPlot.contourf(bx,by,permutedims(b,[2,1]), levels = [-1e5,0], colors = [[.5,.5,.5]])

PyPlot.contourf(gridlonBenthos, gridlatBenthos, permutedims(traits_err, [2,1]), 
    levels = [.5, 1], colors = [[.95,.95,.95]])
ax1[:set_xlim](minimum(gridlonBenthos), maximum(gridlonBenthos))
ax1[:set_ylim](minimum(gridlatBenthos), maximum(gridlatBenthos))

PyCall.PyError: PyError ($(Expr(:escape, :(ccall(#= /home/ctroupin/.julia/packages/PyCall/0jMpb/src/pyfncall.jl:44 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'TypeError'>
TypeError('Dimensions of C (261, 391) are incompatible with X (451) and/or Y (371); see help(pcolormesh)',)
  File "/home/ctroupin/miniconda3/lib/python3.6/site-packages/matplotlib/pyplot.py", line 3245, in pcolormesh
    ret = ax.pcolormesh(*args, **kwargs)
  File "/home/ctroupin/miniconda3/lib/python3.6/site-packages/matplotlib/__init__.py", line 1892, in inner
    return func(ax, *args, **kwargs)
  File "/home/ctroupin/miniconda3/lib/python3.6/site-packages/matplotlib/axes/_axes.py", line 5569, in pcolormesh
    X, Y, C = self._pcolorargs('pcolormesh', *args, allmatch=allmatch)
  File "/home/ctroupin/miniconda3/lib/python3.6/site-packages/matplotlib/axes/_axes.py", line 5192, in _pcolorargs
    C.shape, Nx, Ny, funcname))
